In [6]:
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [159]:
def solve_diffusion(N, T1, T2, x_l, x_r, A, qv, coef_d):
    x = np.linspace(x_l, x_r, N + 1)
    xc = np.array([(x[i] + x[i+1])/2 for i in range(len(x)-1)])
    dx = np.array([x[i+1] - x[i] for i in range(len(x)-1)])
    dxc = np.array([xc[i+1] - xc[i] for i in range(len(xc)-1)])
    dV = A * dx

    aw = np.zeros(N)
    ae = np.zeros(N)
    ap = np.zeros(N)
    Sp = np.zeros(N)
    Su = np.zeros(N)
    F_w = F_e = np.ones_like(dV)
    
    for i in range(0, N):
        
        F_w = rho * C * A * u
        F_e = rho * C * A * u
        
        if(i == 0):
            aw[i] = 0 
            ae[i] = coef_d*A/dxc[i] - F_e/2
            Sp[i] = -2.*coef_d*A/(xc[i] - x_l) - F_w
            ap[i] = aw[i] + ae[i] - Sp[i] + (F_e - F_w)
            Su[i] = T1*2.*coef_d*A/(xc[i] - x_l) + qv*dV[i] - T1 * F_w 
        elif(i == N-1):
            aw[i] = coef_d*A/dxc[i-1] + F_w/2
            ae[i] = 0.
            Sp[i] = -2.*coef_d*A/(x_r - xc[i]) + F_w
            ap[i] = aw[i] + ae[i] - Sp[i] + (F_e - F_w)
            Su[i] = T2*2.*coef_d*A/(x_r - xc[i]) + qv*dV[i] + T2 * F_w
        else:
            aw[i] = coef_d*A/dxc[i-1] + F_w/2  
            ae[i] = coef_d*A/dxc[i] - F_e / 2
            Sp[i] = 0.
            ap[i] = aw[i] + ae[i] - Sp[i] + (F_e - F_w)
            Su[i] = qv*dV[i]
    
    coef_matrix = np.zeros((N, N))
    for i in range(N):
        if(i==0):
            coef_matrix[i][i] = ap[i]
            coef_matrix[i][i+1] = -ae[i]
        elif(i==N-1):
            coef_matrix[i][i] = ap[i]
            coef_matrix[i][i-1] = -aw[i]
        else:
            coef_matrix[i][i+1] = -ae[i]
            coef_matrix[i][i] = ap[i]
            coef_matrix[i][i-1] = -aw[i]
    print (f"Peclet number = {C * rho * u * np.sqrt(A) / coef_d}")
    T = np.linalg.solve(coef_matrix, Su)
    T = np.insert(T, 0, T1)
    T = np.insert(T, N+1, T2)

    x_grid = [x_l] + list(xc) + [x_r]
    return x_grid, T

In [160]:
F_w = rho * C * A * u
F_w

5.0

In [161]:
def solve_analytic(x, T1, T2, x_l, x_r, A, qv, coef_d):
    x = np.array(x)
    return -qv / (2. * coef_d) * (x**2 - x * (x_r + x_l) + x_r * x_l) + (T2 - T1) / (x_r - x_l) * (x - x_l) + T1

In [164]:
T1 = 200 #left boundary
T2 = 300 #right boundary
x_l = 0 #left end
x_r = 1 #right end
A = 0.1 #square
qv = 1000 #flow of heat
coef_d = 10 #teploprovodnost
S_v = 1000 #obiomnii istochnik
u = 0.05 #skorost
C = 100 #teploemkost
rho = 10 #plotnost
N = 100

In [165]:
x, T = solve_diffusion(N, T1, T2, x_l, x_r, A, qv, coef_d)
T_true = solve_analytic(x, T1, T2, x_l, x_r, A, qv, coef_d)

#plt.plot(x, T, lw=2, color='black', label='MFV',zorder=10)
fig = go.Figure()

fig.add_trace(go.Scatter(x = x, y = T, mode = 'markers', name = r'$MFV$', line = dict(color = 'red', width = 1, dash = 'dot')))
fig.add_trace(go.Scatter(x = x, y = T_true, mode = 'markers', name = r'$ideal \, case$', line = dict(color = 'blue', width = 1, dash = 'dot')))
#plt.plot(x, T_true, lw=2, color='blue', ls = '--', label='ideal case', zorder=10)
#plt.scatter(x, T, s=10, color='black', zorder=10)
#plt.grid(zorder=1)
#plt.xlabel('x')
#plt.ylabel('T')
#plt.title('Probe solution')
#plt.legend()
#plt.show()
fig.update_layout(width = 1000, height = 700,
                    margin = dict(l = 10, r = 20, t = 40, b = 20),
                    title = f'Probe solution', title_x = 0.5, title_y = 0.97,
                    xaxis_title = r'$x$',
                    yaxis_title = r'$T$',
                    legend = dict(x = 0.85, y = 0)
  )
fig.show()

Peclet number = 1.5811388300841895


ИЗМЕНЕНА ДЛИНА РАСЧЕТНОЙ ОБЛАСТИ

In [168]:
T1 = 200 #left boundary
T2 = 300 #right boundary
x_l = 0 #left end
x_r = 2 #right end
A = 0.1 #square
qv = 1000 #flow of heat
coef_d = 10 #teploprovodnost
S_v = 1000 #obiomnii istochnik
u = 0.05 #skorost
C = 100 #teploemkost
rho = 1 #plotnost
N = 100

In [169]:
x, T = solve_diffusion(N, T1, T2, x_l, x_r, A, qv, coef_d)
T_true = solve_analytic(x, T1, T2, x_l, x_r, A, qv, coef_d)


fig = go.Figure()

fig.add_trace(go.Scatter(x = x, y = T, mode = 'lines', name = r'$MFV$', line = dict(color = 'red', width = 1, dash = 'solid')))
fig.add_trace(go.Scatter(x = x, y = T_true, mode = 'lines', name = r'$ideal \, case$', line = dict(color = 'blue', width = 1, dash = 'dash')))
fig.update_layout(width = 1000, height = 700,
                    margin = dict(l = 10, r = 20, t = 40, b = 20),
                    title = f'Bigger volume', title_x = 0.5, title_y = 0.97,
                    xaxis_title = r'$x$',
                    yaxis_title = r'$T$',
                    legend = dict(x = 0.85, y = 0)
  )
fig.show()

Peclet number = 0.15811388300841897


ИЗМЕНЕНО ЧИСЛО ЯЧЕЕК

In [170]:
T1 = 200 #left boundary
T2 = 300 #right boundary
x_l = 0 #left end
x_r = 1 #right end
A = 0.1 #square
qv = 1000 #flow of heat
coef_d = 10 #teploprovodnost
S_v = 1000 #obiomnii istochnik
u = 0.05 #skorost
C = 100 #teploemkost
rho = 1 #plotnost

In [171]:
N_list = [5, 10, 20, 50, 100]
colors = ['black', 'red', 'green', 'orange', 'magenta']
fig = go.Figure()



for i in range(len(N_list)):
    x, T = solve_diffusion(N_list[i], T1, T2, x_l, x_r, A, qv, coef_d)
    
    fig.add_trace(go.Scatter(x = x, y = T, mode = 'lines', name = fr'$FVM, N={N_list[i]}$', line = dict(color =colors[i], width = 1, dash = 'solid')))


T_true = solve_analytic(x, T1, T2, x_l, x_r, A, qv, coef_d)

fig.add_trace(go.Scatter(x = x, y = T_true, mode = 'lines', name = r'$ideal \, case$', line = dict(color = 'blue', width = 1, dash = 'dash')))
fig.update_layout(width = 1000, height = 700,
                    margin = dict(l = 10, r = 20, t = 40, b = 20),
                    title = f'Grid convergence', title_x = 0.5, title_y = 0.97,
                    xaxis_title = r'$x$',
                    yaxis_title = r'$T$',
                    legend = dict(x = 0.85, y = 0)
  )
fig.show()

Peclet number = 0.15811388300841897
Peclet number = 0.15811388300841897
Peclet number = 0.15811388300841897
Peclet number = 0.15811388300841897
Peclet number = 0.15811388300841897


ИЗМЕНЕНЫ ПОТОКИ ТЕПЛА

In [172]:
T1 = 200 #left boundary
T2 = 300 #right boundary
x_l = 0 #left end
x_r = 1 #right end
A = 0.1 #square
coef_d = 10 #teploprovodnost
S_v = 1000 #obiomnii istochnik
u = 0.05 #skorost
C = 100 #teploemkost
rho = 10 #plotnost
N = 100

In [173]:
qv_list = [100, 500, 1000, 2000, 5000]
colors = ['black', 'red', 'green', 'orange', 'magenta']
fig = go.Figure()
for i in range(len(qv_list)):
    x, T = solve_diffusion(N, T1, T2, x_l, x_r, A, qv_list[i], coef_d)
    fig.add_trace(go.Scatter(x = x, y = T, mode = 'lines', name = fr'$ FVM, q_v  ={qv_list[i]} $', line = dict(color = colors[i], width = 1, dash = 'solid')))


fig.update_layout(width = 1000, height = 700,
                    margin = dict(l = 10, r = 20, t = 40, b = 20),
                    title = f'Different fluxes', title_x = 0.5, title_y = 0.97,
                    xaxis_title = r'$x$',
                    yaxis_title = r'$T$',
                    legend = dict(x = 0.85, y = 0)
  )
fig.show()

Peclet number = 1.5811388300841895
Peclet number = 1.5811388300841895
Peclet number = 1.5811388300841895
Peclet number = 1.5811388300841895
Peclet number = 1.5811388300841895


ИЗМЕНЕН КОЭФФИЦИЕНТ ТЕМПЕРАТУРОПРОВОДНОСТИ

In [174]:
T1 = 200 #left boundary
T2 = 300 #right boundary
x_l = 0 #left end
x_r = 1 #right end
A = 0.1 #square
qv = 1000 #flow of heat
S_v = 1000 #obiomnii istochnik
u = 0.05 #skorost
C = 100 #teploemkost
rho = 10 #plotnost
N = 100

In [175]:
coef_list = [1, 5, 10, 20, 50]
colors = ['black', 'red', 'green', 'orange', 'magenta']
fig = go.Figure()
for i in range(len(coef_list)):
    x, T = solve_diffusion(N, T1, T2, x_l, x_r, A, qv, coef_list[i]) 
    fig.add_trace(go.Scatter(x = x, y = T, mode = 'lines', name = fr'$ FVM, \lambda={coef_list[i]}$' , line = dict(color = colors[i], width = 1, dash = 'solid')))


fig.update_layout(width = 1000, height = 700,
                    margin = dict(l = 10, r = 20, t = 40, b = 20),
                    title = fr'$Different \,  \lambda$', title_x = 0.5, title_y = 0.97,
                    xaxis_title = r'$x$',
                    yaxis_title = r'$T$',
                    legend = dict(x = 0.85, y = 0)
  )
fig.show()

Peclet number = 15.811388300841896
Peclet number = 3.162277660168379
Peclet number = 1.5811388300841895
Peclet number = 0.7905694150420948
Peclet number = 0.31622776601683794
